In [1]:
import pandas as pd

data = 'RAVEN/TrialRightWrong/Trial'

# All the files we need
files = ['/DEI_trial_by_trial_Right.xlsx', 
         '/DEI_trial_by_trial_Wrong.xlsx', 
         '/esec_trial_by_trial_Right.xlsx',
         '/esec_trial_by_trial_Wrong.xlsx']

In [2]:
for file in files:
    
    # For each file
    f = pd.ExcelFile(data+file)
    
    # Initialize a dict for aggregate the fatigue values for each person and if answer right/wrong a question
    values = {new_list: [] for new_list in f.sheet_names}
    print('File: ',file)
    
    # For each Train
    for sheet in f.sheet_names[1:13]:
        
        df = pd.read_excel(f,  sheet_name=sheet)
        
        # Get the fatigue value for each person
        # In the 'unnamed:22' column is where person id is
        # Notice that if a person isn't in the 'right files', so he is in the 'wrong files'         
        if df.shape[0] > 0:
            for index, row in df[['Unnamed: 22', 'FATIGUE DISPLAY']].iterrows():

                # cell = [person, fatigue_value, correct(1)/incorret(0)]
                cell = []
                cell += [row[0], row[1]]
                cell += [1 if file == ('/DEI_trial_by_trial_Right.xlsx' or '/esec_trial_by_trial_Right.xlsx') else 0]
                values[sheet] += [cell]
        
        print("Folha: ",sheet)
        print(values[sheet])
        print('-------------------------------------------')        
            

File:  /DEI_trial_by_trial_Right.xlsx
Folha:  TRIAL TRAINING1
[['DEI_02', 1.5125678693069673, 1], ['DEI_03', 0.51180466643057, 1], ['DEI_04', 0.18773382821972204, 1], ['DEI_05', 1.3319469592154476, 1], ['DEI_06', 0.22073851828511953, 1], ['DEI_08', 0.37523472496623206, 1], ['DEI_09', 0.9593306853136819, 1], ['DEI_10', 1.0224084920827694, 1], ['DEI_11', 0.8724738868355004, 1], ['DEI_12', 2.780199167086349, 1], ['DEI_13', 0.6923004277578605, 1], ['DEI_14', 0.8558587663320589, 1], ['DEI_15', 0.8465007523496654, 1], ['DEI_16', 1.6443068579584734, 1], ['DEI_17', 1.3385806672729654, 1], ['DEI_18', 3.0538805855078537, 1], ['DEI_19', 2.0680193036125605, 1], ['DEI_20', 0.6894987854700674, 1], ['DEI_21', 1.9746477456600044, 1], ['DEI_22', 0.945235948591516, 1], ['DEI_23', 8.468752234960249, 1], ['DEI_24', 5.491672784531988, 1]]
-------------------------------------------
Folha:  TRIAL TRAINING2
[['DEI_01', 4.915042293066063, 1], ['DEI_02', 0.5128177118739389, 1], ['DEI_03', 0.9122285779879463, 1

File:  /esec_trial_by_trial_Right.xlsx
Folha:  TRIAL TRAINING1
[['ESEC_01', 0.9981191880417031, 0], ['ESEC_02', 0.7508528142773631, 0], ['ESEC_04', 1.4759835695827928, 0], ['ESEC_07', 3.3358450743944505, 0], ['ESEC_09', 0.6264716371091513, 0], ['ESEC_10', 0.3986654584599284, 0], ['ESEC_11', 15.276921727584057, 0], ['ESEC_12', 1.1681538450181934, 0], ['ESEC_15', 0.5100365056412502, 0], ['ESEC_16', 0.9782086331579603, 0], ['ESEC_17', 2.410128550534691, 0], ['ESEC_18', 5.210418549673369, 0], ['ESEC_20', 0.5870825128734974, 0]]
-------------------------------------------
Folha:  TRIAL TRAINING2
[['ESEC_01', 1.2554089880999166, 0], ['ESEC_02', 1.6415309619014733, 0], ['ESEC_03', 0.94953608149745, 0], ['ESEC_04', 0.7796546523689011, 0], ['ESEC_05', 0.6567846217879493, 0], ['ESEC_07', 1.0094083241783307, 0], ['ESEC_08', 0.3669513355913879, 0], ['ESEC_09', 0.6660714582950797, 0], ['ESEC_10', 1.4073367504281333, 0], ['ESEC_11', 11.837756749667832, 0], ['ESEC_12', 0.23204133223382134, 0], ['ESEC